In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bertopic import BERTopic
from get_title import *
from tqdm import tqdm

2023-08-20 13:02:31.982830: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-20 13:02:31.992071: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-20 13:02:32.027550: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-20 13:02:32.028505: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 13:02:32.569556: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
visits = pd.read_csv('../tables/visits.csv')
urls = pd.read_csv('../tables/urls.csv')
visits.head()

,id,url,visit_time,from_visit,visit_duration
0,55623,19110,2023-08-17 08:43:58,0,509
1,55622,19120,2023-08-17 08:38:14,55621,852
2,55621,19119,2023-08-17 08:38:13,0,1
3,55620,19119,2023-08-17 08:38:12,0,0
4,55619,19118,2023-08-17 08:30:57,55618,1290


In [3]:
n_topics = 10

In [16]:
P = [[0]*n_topics for _ in range(n_topics)]
P

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [17]:
transitions = []
to_url = None
for idx, row in visits.iterrows():
    if row[3]==0:
        if to_url:
            transitions.append([to_url, row[1]])
        continue
    to_url = row[1]
    from_url = visits.query(f"id=={row[3]}").url.item()
    if from_url != to_url:
        transitions.append([from_url, to_url])

In [10]:
len(transitions)

5080

In [11]:
url = urls.query(f"id=={transitions[0][1]}").url.item()
print(url)
topic = get_topic(url)[0][0]
topic

https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html


4

In [12]:
P = [[0]*n_topics for _ in range(n_topics)]
for idx, t in enumerate(transitions):
    topic1, topic2 = get_topic(t[0], True), get_topic(t[1], True)
    if topic1 and topic2:
        P[topic1[0][0]][topic2[0][0]] += 1

In [13]:
P

[[217, 1, 73, 5, 1, 1, 5, 1, 0, 25],
 [1, 774, 6, 141, 2, 0, 0, 0, 0, 17],
 [87, 9, 1404, 75, 132, 26, 18, 90, 12, 120],
 [3, 121, 40, 550, 6, 0, 1, 0, 1, 41],
 [1, 1, 10, 2, 47, 0, 0, 0, 0, 1],
 [1, 2, 8, 2, 1, 75, 3, 0, 0, 9],
 [1, 1, 29, 2, 1, 1, 52, 0, 0, 7],
 [0, 0, 10, 1, 1, 0, 0, 44, 2, 3],
 [0, 1, 4, 0, 0, 1, 0, 1, 5, 0],
 [32, 75, 159, 127, 5, 8, 16, 10, 1, 311]]

In [14]:
for idx, row in enumerate(P):
    total = sum(row)
    for idx, _ in enumerate(row):
        row[idx] = round(row[idx]/total, 2)

In [15]:
P

[[0.66, 0.0, 0.22, 0.02, 0.0, 0.0, 0.02, 0.0, 0.0, 0.08],
 [0.0, 0.82, 0.01, 0.15, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02],
 [0.04, 0.0, 0.71, 0.04, 0.07, 0.01, 0.01, 0.05, 0.01, 0.06],
 [0.0, 0.16, 0.05, 0.72, 0.01, 0.0, 0.0, 0.0, 0.0, 0.05],
 [0.02, 0.02, 0.16, 0.03, 0.76, 0.0, 0.0, 0.0, 0.0, 0.02],
 [0.01, 0.02, 0.08, 0.02, 0.01, 0.74, 0.03, 0.0, 0.0, 0.09],
 [0.01, 0.01, 0.31, 0.02, 0.01, 0.01, 0.55, 0.0, 0.0, 0.07],
 [0.0, 0.0, 0.16, 0.02, 0.02, 0.0, 0.0, 0.72, 0.03, 0.05],
 [0.0, 0.08, 0.33, 0.0, 0.0, 0.08, 0.0, 0.08, 0.42, 0.0],
 [0.04, 0.1, 0.21, 0.17, 0.01, 0.01, 0.02, 0.01, 0.0, 0.42]]

In [43]:
P = np.array(P)
print(P)
print("\n\n")
print(P @ P)
print("\n\n")
print(P @ P @ P)
print("\n\n")
print(P @ P @ P @ P)
print("\n\n")

[[0.52 0.06 0.12 0.26 0.   0.01 0.01 0.01 0.  ]
 [0.1  0.54 0.08 0.25 0.   0.01 0.01 0.   0.  ]
 [0.19 0.08 0.45 0.19 0.02 0.01 0.03 0.02 0.01]
 [0.09 0.19 0.14 0.56 0.01 0.   0.   0.   0.  ]
 [0.05 0.02 0.24 0.07 0.56 0.01 0.01 0.04 0.  ]
 [0.45 0.05 0.1  0.19 0.   0.18 0.02 0.   0.  ]
 [0.32 0.08 0.17 0.23 0.   0.   0.19 0.01 0.  ]
 [0.29 0.03 0.21 0.22 0.01 0.   0.   0.22 0.01]
 [0.19 0.22 0.18 0.3  0.   0.01 0.   0.01 0.1 ]]



[[0.3332 0.1242 0.1624 0.325  0.0051 0.0088 0.0115 0.0099 0.0013]
 [0.1514 0.3528 0.1289 0.3204 0.0041 0.009  0.0109 0.0027 0.0008]
 [0.2322 0.1328 0.2752 0.2789 0.0223 0.0093 0.0223 0.0165 0.0057]
 [0.1433 0.2258 0.1698 0.4118 0.014  0.0043 0.0071 0.0041 0.0014]
 [0.1272 0.06   0.2709 0.155  0.3195 0.0105 0.0156 0.0366 0.0028]
 [0.3625 0.1087 0.151  0.2937 0.0039 0.0384 0.0154 0.0067 0.001 ]
 [0.2911 0.1352 0.1879 0.3102 0.0058 0.0057 0.0452 0.0107 0.0018]
 [0.2797 0.1012 0.2129 0.2981 0.0142 0.0055 0.0096 0.056  0.0053]
 [0.2084 0.2244 0.1845 0.3407 0.0067